In [1]:
import tensorflow as tf
import numpy as np
import importlib
from train import provider

/home/vamsi/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import cPickle as cPickle
def demo(fname):
    #import mayavi.mlab as mlab
    #from viz_util import draw_lidar, draw_lidar_simple, draw_gt_boxes3d
    with open(fname, 'rb') as input_file:
        try:
            while True:
                yield cPickle.load(input_file)
        except EOFError:
            pass

In [ ]:
h = list(demo("./kitti/frustum_carpedcyc_train.pickle"))

KeyboardInterrupt: 

In [ ]:
val = list(demo("./kitti/frustum_carpedcyc_val.pickle"))

In [ ]:
NUM_POINT=2048
TRAIN_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='train',\
                            rotate_to_center=True, random_flip=False, random_shift=True, one_hot=True)

In [ ]:
VAL_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='val',\
                            rotate_to_center=True, random_flip=False, random_shift=True, one_hot=True)

In [ ]:

def indices(num, window_len, stride_len, key='train'):
    IOU = []
    start = []
    end = []
    labels = []
    point_set = []
    if key=='train':
        ind = np.argsort(TRAIN_DATASET.get_center_view_point_set(num)[:,2])
        point_set = (TRAIN_DATASET.get_center_view_point_set(num)[ind])[:,:3]
        labels = h[4][num]
        labels = labels[ind]
    elif key=='val':
        ind = np.argsort(VAL_DATASET.get_center_view_point_set(num)[:,2])
        point_set = (VAL_DATASET.get_center_view_point_set(num)[ind])[:,:3]
        labels = val[4][num]
        labels = labels[ind]
    j = np.min(point_set,0)[2]
    i = np.min(point_set,0)[2]+window_len
    idtuple=set()
    while i<=np.max(point_set,0)[2]:
            max_ind = np.searchsorted(point_set[:,2], i)
            min_ind = np.searchsorted(point_set[:,2], j)
            if (min_ind,max_ind) not in idtuple and max_ind-min_ind>5:
                #print('fgbd')
                idtuple.add((min_ind,max_ind))
                labels_window = np.zeros_like(point_set[:,0])
                labels_window[min_ind:max_ind] = 1.0
                union = np.sum(np.logical_or(labels, labels_window))
                inter = np.sum(np.logical_and(labels, labels_window))
                if (inter*1.0/union) < 0.3 or (inter*1.0/union) >0.7 :
                    end.append(max_ind)
                    start.append(min_ind)
                    IOU.append(inter*1.0/union)
            j = j+stride_len
            i = j+window_len
    IOU = np.array(IOU)>0.7
    return np.array(start),np.array(end), IOU

a,b,c = indices(0,1.0,0.1)
a.shape,b.shape,c.shape

In [ ]:
# points in each window:2048
# number of windows
#

def get_list(start_idx,end_idx,num_points=1024,ensure_all=True):
    """
    param:
    start_idx  : list of indexes of the starting point in each window
    end_idx    : list of indees of ending point in each window
    NOTES: we used two lists because we ran into issues with tensorflow when we used tuples,
           this is a quick hack to avoid that
    num_points : number of points to be sampled from i.e number of points in each window
    ensure_all : in the case that there are fewwer than required points ensures all points are picked atleast once if selected as True
    
    
    returns:
    out:  an array of size 
          
          num_windowsxnum_pointsx1
          
          this will be used inside tensorflow to divide into windows 
          after we pass the points trough pointnet
    """
    
    out= []
    for i in range(np.shape(start_idx)[0]):
        
        if end_idx[i]-start_idx[i]+1<num_points:
            indexes=np.arange(start_idx[i],end_idx[i]+1)
            indexes2=np.random.choice(indexes, num_points-np.shape(indexes)[0],replace=True)
            indexes=np.concatenate((indexes,indexes2))
        else:
            indexes=np.random.choice(np.arange(start_idx[i],end_idx[i]+1),num_points,replace=False)
        out.append(np.reshape(indexes,(-1,1)))
    
    return np.array(out)


In [ ]:
#check
#for i in range(len(h[5])):
s,e,l=indices(0, 0.9, 0.1)

s1, e1, l1 = indices(0, 3.0, 0.3)

ind= get_list(s,e, num_points=512)

ind1 = get_list(s1,e1, num_points=1024)

In [ ]:
print(ind.shape)
print(ind1.shape)

In [ ]:
import tf_util #for conv2d
from model_util import NUM_HEADING_BIN, NUM_SIZE_CLUSTER, NUM_OBJECT_POINT
from model_util import point_cloud_masking, get_center_regression_net
from model_util import placeholder_inputs, parse_output_to_tensors, get_loss


In [ ]:
#def focal_loss(prediction,target,gamma,alpha):
    
    
    

In [ ]:
# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py


from tensorflow.python.ops import array_ops

def focal_loss(sigmoid_p, target_tensor, weights=None, alpha=0.99, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log
            (1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    #sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(target_tensor, dtype=target_tensor.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1.0 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

In [ ]:
# def classification_net(point_cloud, label_out_5, label_out_1, label_out_2, label_out_3, label_out_4, idx5, idx1, idx2, idx3, idx4, 
#                             is_training=True, bn_decay=True):
def classification_net(point_cloud, label_out_5,  idx5, 
                            is_training=True, bn_decay=True):
    '''
    3D PointNet network.
    param:
    
    point_cloud: TF tensor in shape (1,N,4)
                 frustum point clouds with XYZ and intensity in point channels
                 XYZs are in frustum coordinate
    label_out: TF tensor in shape (1,1)
            length-1 vector indicating the classification as car or background
    
    ??????????????????????????????????????????????????????????????????????????????????????
    
    is_training: TF boolean scalar
    bn_decay: TF float scalar
    end_points: dict                 SHOULD CHANGE THIS
    Output:                                         
    logits: TF tensor in shape (B,N,2), scores for bkg/clutter and object
    end_points: dict
    ??????????????????????????????????????????????????????????????????????????????????????
    '''
    
    batch_size = point_cloud.get_shape()[0].value #1 in our case
    num_point = point_cloud.get_shape()[1].value  #N in our case
  
    net = tf.expand_dims(point_cloud, 2)
    #print(net.get_shape())

    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv2', bn_decay=bn_decay)
    point_feat = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv3', bn_decay=bn_decay)
    net = tf_util.conv2d(point_feat, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv4', bn_decay=bn_decay)
    features = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv5', bn_decay=bn_decay)
    
    features=tf.squeeze(features,axis=2)
    features=tf.squeeze(features,axis=0)   

    #features=tf.placeholder(tf.float32,[1,1024,4])

    def fn(x):
        return tf.gather_nd(features,x)
    
    windows_5 =tf.map_fn(fn,idx5,dtype=tf.float32)
#     windows_1 = tf.map_fn(fn,idx1,dtype=tf.float32)
#     windows_2 = tf.map_fn(fn,idx2,dtype=tf.float32)
#     windows_3 = tf.map_fn(fn,idx3,dtype=tf.float32)
#     windows_4 = tf.map_fn(fn,idx4,dtype=tf.float32)
    
    windows_5_1024 = tf.reduce_max(windows_5,axis=1)
#     windows_1_1024 = tf.reduce_max(windows_1,axis=1)
#     windows_2_1024 = tf.reduce_max(windows_2,axis=1)
#     windows_3_1024 = tf.reduce_max(windows_3,axis=1)
#     windows_4_1024 = tf.reduce_max(windows_4,axis=1)
    
    
    winex_5 =  tf.reshape(windows_5_1024,shape=[-1,1,1,1024])
#     winex_1 =  tf.reshape(windows_1_1024,shape=[-1,1,1,1024])
#     winex_2 =  tf.reshape(windows_2_1024,shape=[-1,1,1,1024])
#     winex_3 =  tf.reshape(windows_3_1024,shape=[-1,1,1,1024])
#     winex_4 =  tf.reshape(windows_4_1024,shape=[-1,1,1,1024])
   
    winex = tf.concat([winex_5], axis=0)
    label_out = tf.concat([label_out_5],axis=0)

    
    
#     winex = tf.concat([winex_5, winex_1, winex_2, winex_3, winex_4], axis=0)
#     label_out = tf.concat([label_out_5,label_out_1, label_out_2, label_out_3, label_out_4],axis=0)

    #classification head
    d1 = tf_util.conv2d(winex, 512, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='dense1', bn_decay=bn_decay)
    d2 = tf_util.conv2d(d1, 256, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='dense2', bn_decay=bn_decay)
    d2=tf.layers.dropout(d2,rate=0.5, training=is_training, name='drop1')
    
    logits = tf_util.conv2d(d2, 1, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='out', bn_decay=bn_decay, activation_fn=None)
    
    logits = tf.squeeze(logits)
    logits=tf.reshape(tf.squeeze(logits),(1,-1))
    sig = tf.sigmoid(logits)

    #sig = tf.reshape(tf.squeeze(sig),(-1,1))
    y=tf.reshape(sig,[-1])
    
    max_ind = tf.argmax(y)
    
    tp = tf.equal(label_out[max_ind], 1)
    
    
    differentiable_round = tf.maximum(y-0.499,0)
    differentiable_round = differentiable_round * 10000
    y_pred = tf.minimum(differentiable_round, 1)
    #y_pred=tf.round(y)
    
    true_pos  = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(label_out,tf.bool),tf.cast(y_pred,tf.bool)),tf.float32))
    
    false_pos = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(tf.cast(label_out,tf.bool)),tf.cast(y_pred,tf.bool)),tf.float32))
    true_neg  = tf.reduce_sum(tf.cast(tf.logical_not(tf.logical_or(tf.cast(label_out, tf.bool),\
                                                                   tf.cast(y_pred, tf.bool))),tf.float32))
    false_neg = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not\
                                                     (tf.cast(y_pred,tf.bool)),tf.cast(label_out,tf.bool)),tf.float32))
    return sig,logits, label_out,true_pos,false_pos,true_neg,false_neg, tp, max_ind

In [ ]:
# ch=tf.reshape(sig,[-1])
# ch=tf.round(ch)

In [ ]:
# ch.get_shape()

In [ ]:
tf.reset_default_graph()
with tf.device('/device:GPU:0'):
    num_points_5 =512
#     num_points_1 = 512
#     num_points_2 = 512
#     num_points_3 = 512
#     num_points_4 = 512
    
    inputs=tf.placeholder(tf.float32,[None,2048,4])#put placeholder

    idx_5 =tf.placeholder(tf.int32,[None,num_points_5,1])

#     idx_1 = tf.placeholder(tf.int32,[None,num_points_1,1])
    
#     idx_2 = tf.placeholder(tf.int32,[None,num_points_2,1])

#     idx_3 = tf.placeholder(tf.int32,[None,num_points_3,1])
    
#     idx_4 = tf.placeholder(tf.int32,[None,num_points_4,1])

    is_training = tf.placeholder(tf.bool)
    

    label_out_5 = tf.placeholder(tf.int32, [None])
#     label_out_1 = tf.placeholder(tf.int32, [None])
#     label_out_2 = tf.placeholder(tf.int32, [None])
#     label_out_3 = tf.placeholder(tf.int32, [None])
#     label_out_4 = tf.placeholder(tf.int32, [None])

    sig,logits, label_out,tp,fp,tn,fn, tp_num, maxi = classification_net(inputs, label_out_5, \
                                         idx_5, is_training)
    
    
#     sig,logits, label_out,tp,fp,tn,fn = classification_net(inputs, label_out_5, label_out_1,label_out_2, label_out_3, label_out_4, \
#                                          idx_5, idx_1, idx_2, idx_3, idx_4, is_training)
    
    func_fl = focal_loss(tf.squeeze(sig),tf.squeeze(tf.cast(label_out, tf.float32)), gamma=0.75, alpha=0.25)
    print(label_out.get_shape())
#     loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#     labels=tf.reshape(tf.cast(label_out,tf.float32),(-1,1)),
#     logits=tf.reshape(logits,(-1,1))))

    loss = tf.reduce_mean(func_fl)
#     tpr=tf.divide(tf.cast(tp,tf.float32),tf.add(tp,fp))
#     tnr=tf.divide(tf.cast(tn,tf.float32),tf.add(tn,fn))
#     loss=tf.reduce_sum(tf.constant(1.0)-tf.scalar_mul(0.5,tf.add(tpr,tnr)))
    
    
    optimizer = tf.train.AdamOptimizer(1e-5)
    #train_step=optimizer.minimize(loss)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)
    
    

In [ ]:
from tensorflow.python import debug

In [ ]:
import datetime
import logging
from random import shuffle
#logging.basicConfig(filename='run_1.log',level=logging.DEBUG)
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))

saver = tf.train.Saver()
trix=[cc for cc in range(len(h[5]))]
shuffle(trix)
prob = []
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    #sess=tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print "Variable: ", k
        print "Shape: ", v.shape
    # uncomment this when we want to load the checkpoint model
    saver.restore(sess, "./checkpoints/model_Gamma0.75_14_8666.ckpt")
    
    
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    k = 0
    print(len(val[5]))
    Vtotal_tp=0
    Vtotal_fp=0
    Vtotal_tn=0
    Vtotal_fn=0
    Vtotal_tpnums = 0
    Vtotal_fpnums = 0
    Vtotal_fnnums = 0
    while k<len(val[5]):
    #todo reformat code to have lists instead of many variable


        if val[5][k]=='Pedestrian':

            index = np.argsort(VAL_DATASET.get_center_view_point_set(k)[:,2])
            point_set = VAL_DATASET.get_center_view_point_set(k)[index]

            if point_set.shape[0]<2048:
                    indexes=np.arange(np.shape(point_set)[0])
                    indexes2=np.random.choice(indexes, 2048-np.shape(indexes)[0],replace=True)
                    indexes=np.concatenate((indexes,indexes2))
            else:
                    indexes=np.random.choice(np.arange(np.shape(point_set)[0]),2048,replace=False)

            point_set = np.reshape(point_set[indexes], (1, 2048, 4))

            s5,e5,l5 = indices(k, 0.9, 0.1, 'val')

#                 s1, e1, l1 = indices(k, 1.1, 0.2, 'val')

#                 s2, e2, l2 = indices(k, 1.3, 0.2, 'val')

#                 s3, e3, l3 = indices(k, 0.7, 0.1, 'val')

#                 s4, e4, l4 = indices(k, 0.5, 0.1, 'val')

            if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)

#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                print(Vtotal_tp)

                x,tpos,fpos,tneg,fneg, tp_nums, maxind =sess.run([loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5, \
                                                           label_out_5: l5.astype(np.int32), inputs:point_set, is_training: True})
                print(Vtotal_tp)
                Vtotal_tp +=tpos
                Vtotal_fp +=fpos
                Vtotal_tn +=tneg
                Vtotal_fn +=fneg
                Vtotal_tpnums+= tp_nums
                Vtotal_fpnums+= (1-tp_nums)
                Vtotal_fnnums+= (1-tp_nums)
                
                print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
                
                print('Debug :',maxind, l5[maxind], tp_nums)
                print('Epoch ', j , 'Loss for the current example ',k,' = ', x)
                #logging.info('Loss for the current example ',i,' = ', x)
                print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                print('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))

                print('Single metrics: Precision = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fpnums),\
                      ' Recall = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fnnums))
                #logging.info('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                #      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

        k += 1

    
    
    
    
    
    
    for j in range(6,15):   
        total_tp=0
        total_fp=0
        total_tn=0
        total_fn=0
        total_tpnums = 0
        total_fpnums = 0
        total_fnnums = 0
        
        
        print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
        
        print('Epoch Number........................................',j)
        pedc=0
        
        for i in trix:
            
        #todo reformat code to have lists instead of many variable
            if h[5][i]=='Pedestrian':
                pedc+=1
                if pedc%500==0:
                    total_tp=0
                    total_fp=0
                    total_tn=0
                    total_fn=0
                index = np.argsort(TRAIN_DATASET.get_center_view_point_set(i)[:,2])
                point_set = TRAIN_DATASET.get_center_view_point_set(i)[index]

                if point_set.shape[0]<2048:
                        indexes=np.arange(np.shape(point_set)[0])
                        indexes2=np.random.choice(indexes, 2048-np.shape(indexes)[0],replace=True)
                        indexes=np.concatenate((indexes,indexes2))
                else:
                        indexes=np.random.choice(np.arange(np.shape(point_set)[0]),2048,replace=False)

                point_set = np.reshape(point_set[indexes], (1, 2048, 4))
                
                
                
                s5,e5,l5 = indices(i, 0.9, 0.1)

#                 s1, e1, l1 = indices(i, 1.1, 0.2)

#                 s2, e2, l2 = indices(i, 1.3, 0.2)

#                 s3, e3, l3 = indices(i, 0.7, 0.1)

#                 s4, e4, l4 = indices(i, 0.5, 0.1)
                
                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)
#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]
                        
                    _, x,tpos,fpos,tneg,fneg, tpnums =sess.run([train_op, loss,tp,fp,tn,fn, tp_num],feed_dict={idx_5:ind_5,\
                                                               label_out_5: l5.astype(np.int32),\
                                                               inputs:point_set, is_training: True})
                    total_tp+=tpos
                    total_fp+=fpos
                    total_tn+=tneg
                    total_fn+=fneg
                    total_tpnums+= tpnums
                    total_fpnums+= (1-tpnums)
                    total_fnnums+= (1-tpnums)
                    print('----------------------------------------------------------------------------------')
                    print('Epoch ', j , 'Loss for the current example ',i,' = ', x,'pedc = ',pedc)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                          ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    #logging.info('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                    #      ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    print('Single metrics: Precision = ',(total_tpnums*1.0)/(total_tpnums+total_fpnums),\
                          ' Recall = ',(total_tpnums*1.0)/(total_tpnums+total_fnnums))
                    print('----------------------------------------------------------------------------------')
                    
                    if(pedc%5000==0):
                        save_path = saver.save(sess, "./checkpoints/model_Gamma0.75_"+str(j)+'_'+str(i)+".ckpt")
                    if pedc%12000==11999:
                        print('***************************************************************************************************')
                        k = 0
                        print(len(val[5]))
                        Vtotal_tp=0
                        Vtotal_fp=0
                        Vtotal_tn=0
                        Vtotal_fn=0
                        while k<len(val[5]):
                        #todo reformat code to have lists instead of many variable
                            
                        
                            if val[5][k]=='Pedestrian':
                                
                                index = np.argsort(VAL_DATASET.get_center_view_point_set(k)[:,2])
                                point_set = VAL_DATASET.get_center_view_point_set(k)[index]
                                
                                if point_set.shape[0]<2048:
                                        indexes=np.arange(np.shape(point_set)[0])
                                        indexes2=np.random.choice(indexes, 2048-np.shape(indexes)[0],replace=True)
                                        indexes=np.concatenate((indexes,indexes2))
                                else:
                                        indexes=np.random.choice(np.arange(np.shape(point_set)[0]),2048,replace=False)
                                
                                point_set = np.reshape(point_set[indexes], (1, 2048, 4))

                                s5,e5,l5 = indices(k, 0.9, 0.1, 'val')

#                                 s1, e1, l1 = indices(k, 1.1, 0.2, 'val')

#                                 s2, e2, l2 = indices(k, 1.3, 0.2, 'val')

#                                 s3, e3, l3 = indices(k, 0.7, 0.1, 'val')

#                                 s4, e4, l4 = indices(k, 0.5, 0.1, 'val')
                                
                                if s5.size :#and s2.size and s3.size and s4.size and s1.size:
                                    
                                    ind_5 = get_list(s5,e5, num_points=512)

#                                     ind_1 = get_list(s1,e1, num_points=512)

#                                     ind_2 = get_list(s2,e2, num_points=512)

#                                     ind_3 = get_list(s3,e3, num_points=512)

#                                     ind_4 = get_list(s4,e4, num_points=512)

#                                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]
                                    
                                    print(Vtotal_tp)

                                    x,tpos,fpos,tneg,fneg =sess.run([loss,tp,fp,tn,fn],feed_dict={idx_5:ind_5, \
                                                                               label_out_5: l5.astype(np.int32), inputs:point_set, is_training : True})
                                    
                                    print(Vtotal_tp)
                                    Vtotal_tp +=tpos
                                    Vtotal_fp +=fpos
                                    Vtotal_tn +=tneg
                                    Vtotal_fn +=fneg
                                    print('***************************************************************************************************')
                                    print('Epoch ', j , 'Loss for the current example ',k,' = ', x)
                                    #logging.info('Loss for the current example ',i,' = ', x)
                                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                                    print('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                                          ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                                    #logging.info('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                                    #      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                                    print('***************************************************************************************************')
                                    
                            k += 1

                        
                        

                #print(c)
                #print(n)
            #i += 1
        #-----------------------------------------------------------------------------------------------#
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
        k = 0
        print(len(val[5]))
        Vtotal_tp=0
        Vtotal_fp=0
        Vtotal_tn=0
        Vtotal_fn=0
        Vtotal_tpnums = 0
        Vtotal_fpnums = 0
        Vtotal_fnnums = 0
        while k<len(val[5]):
        #todo reformat code to have lists instead of many variable


            if val[5][k]=='Pedestrian':

                index = np.argsort(VAL_DATASET.get_center_view_point_set(k)[:,2])
                point_set = VAL_DATASET.get_center_view_point_set(k)[index]

                if point_set.shape[0]<2048:
                        indexes=np.arange(np.shape(point_set)[0])
                        indexes2=np.random.choice(indexes, 2048-np.shape(indexes)[0],replace=True)
                        indexes=np.concatenate((indexes,indexes2))
                else:
                        indexes=np.random.choice(np.arange(np.shape(point_set)[0]),2048,replace=False)

                point_set = np.reshape(point_set[indexes], (1, 2048, 4))

                s5,e5,l5 = indices(k, 0.9, 0.1, 'val')

#                 s1, e1, l1 = indices(k, 1.1, 0.2, 'val')

#                 s2, e2, l2 = indices(k, 1.3, 0.2, 'val')

#                 s3, e3, l3 = indices(k, 0.7, 0.1, 'val')

#                 s4, e4, l4 = indices(k, 0.5, 0.1, 'val')

                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)

#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                    print(Vtotal_tp)

                    x,tpos,fpos,tneg,fneg, tp_nums =sess.run([loss,tp,fp,tn,fn, tp_num],feed_dict={idx_5:ind_5, \
                                                               label_out_5: l5.astype(np.int32), inputs:point_set, is_training: True})
                    print(Vtotal_tp)
                    Vtotal_tp +=tpos
                    Vtotal_fp +=fpos
                    Vtotal_tn +=tneg
                    Vtotal_fn +=fneg
                    Vtotal_tpnums+= tpnums
                    Vtotal_fpnums+= (1-tpnums)
                    Vtotal_fnnums+= (1-tpnums)
                    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
                    print('Epoch ', j , 'Loss for the current example ',k,' = ', x)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                          ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                    
                    print('Single metrics: Precision = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fpnums),\
                          ' Recall = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fnnums))
                    #logging.info('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                    #      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

            k += 1

        #----------------------------------------------------------------------------------------------#
#         prob = sess.run([sig],feed_dict={idx_5:ind_5, idx_1:ind_1, idx_2:ind_2, idx_3:ind_3, idx_4:ind_4,\
#                                                        label_out_5: l5.astype(np.int32), label_out_1: l1.astype(np.int32),\
#                                                        label_out_2: l2.astype(np.int32), label_out_3: l3.astype(np.int32),\
#                                                        label_out_4: l4.astype(np.int32), inputs:point_set,  is_training: False})
#         print(prob)
        print(ind_5)
        print(label_out_5)
        prob = sess.run([sig],feed_dict={idx_5:ind_5,  \
                                                       label_out_5: l5.astype(np.int32), inputs:point_set,  is_training: False})
        print(prob)
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S") )

In [ ]:
def classification_net(point_cloud, label_out_5, label_out_1, label_out_2, label_out_3, label_out_4, idx5, idx1, idx2, idx3, idx4, 
                            is_training=True, bn_decay=None):
    '''
    3D PointNet network.
    param:
    
    point_cloud: TF tensor in shape (1,N,4)
                 frustum point clouds with XYZ and intensity in point channels
                 XYZs are in frustum coordinate
    label_out: TF tensor in shape (1,1)
            length-1 vector indicating the classification as car or background
    
    ??????????????????????????????????????????????????????????????????????????????????????
    
    is_training: TF boolean scalar
    bn_decay: TF float scalar
    end_points: dict                 SHOULD CHANGE THIS
    Output:                                         
    logits: TF tensor in shape (B,N,2), scores for bkg/clutter and object
    end_points: dict
    ??????????????????????????????????????????????????????????????????????????????????????
    '''
    
    batch_size = point_cloud.get_shape()[0].value #1 in our case
    num_point = point_cloud.get_shape()[1].value  #N in our case
  
    net = tf.expand_dims(point_cloud, 2)
    #print(net.get_shape())
    print('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
    net = tf.layers.conv2d(net, 64, [1,1], strides=[1,1], padding='valid', name='conv1', activation= tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    net = tf.layers.batch_normalization(net, training=is_training, name='bn1')
    
    net = tf.layers.conv2d(net, 64, [1,1], strides=[1,1], padding='valid', name='conv2', activation= tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    net = tf.layers.batch_normalization(net, training=is_training, name='bn2')
    
    net = tf.layers.conv2d(net, 64, [1,1], strides=[1,1], padding='valid', name='conv3', activation= tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    net = tf.layers.batch_normalization(net, training=is_training, name='bn3')
    
    net = tf.layers.conv2d(net, 128, [1,1], strides=[1,1], padding='valid', name='conv4', activation= tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    net = tf.layers.batch_normalization(net, training=is_training, name='bn4')
    
    net = tf.layers.conv2d(net, 1024, [1,1], strides=[1,1], padding='valid', name='conv5', activation= tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())
    print('BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB')
    features = tf.layers.batch_normalization(net, training=is_training, name='bn5')   

    
    features=tf.squeeze(features,axis=2)
    features=tf.squeeze(features,axis=0)   

    #features=tf.placeholder(tf.float32,[1,1024,4])

    def fn(x):
        return tf.gather_nd(features,x)
    
    windows_5 =tf.map_fn(fn,idx5,dtype=tf.float32)
    windows_1 = tf.map_fn(fn,idx1,dtype=tf.float32)
    windows_2 = tf.map_fn(fn,idx2,dtype=tf.float32)
    windows_3 = tf.map_fn(fn,idx3,dtype=tf.float32)
    windows_4 = tf.map_fn(fn,idx4,dtype=tf.float32)
    
    windows_5_1024 = tf.reduce_max(windows_5,axis=1)
    windows_1_1024 = tf.reduce_max(windows_1,axis=1)
    windows_2_1024 = tf.reduce_max(windows_2,axis=1)
    windows_3_1024 = tf.reduce_max(windows_3,axis=1)
    windows_4_1024 = tf.reduce_max(windows_4,axis=1)
    
    
    winex_5 =  tf.reshape(windows_5_1024,shape=[-1,1,1,1024])
    winex_1 =  tf.reshape(windows_1_1024,shape=[-1,1,1,1024])
    winex_2 =  tf.reshape(windows_2_1024,shape=[-1,1,1,1024])
    winex_3 =  tf.reshape(windows_3_1024,shape=[-1,1,1,1024])
    winex_4 =  tf.reshape(windows_4_1024,shape=[-1,1,1,1024])
    
    winex = tf.concat([winex_5, winex_1, winex_2, winex_3, winex_4], axis=0)
    label_out = tf.concat([label_out_5,label_out_1, label_out_2, label_out_3, label_out_4],axis=0)
    print('CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC')
    conv = tf.contrib.layers.fully_connected(winex, 512, scope='dense1')

    conv = tf.layers.batch_normalization(conv, training=is_training, name='bn6')

    conv = tf.contrib.layers.fully_connected(conv, 128, scope='dense2')

    conv = tf.layers.dropout(conv,rate=0.5, training=is_training, name='drop1')
    
    conv = tf.layers.batch_normalization(conv, training=is_training, name='bn7')
    print('DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD')
    logits = tf.contrib.layers.fully_connected(conv, 1, scope='dense3')
    
    logits = tf.squeeze(logits)
    logits = tf.reshape(tf.squeeze(logits),(-1,1))
    sig = tf.sigmoid(logits)
    #sig = tf.reshape(tf.squeeze(sig),(-1,1))
    y=tf.reshape(sig,[-1])
    y_pred=tf.round(y)
    print('EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE')
    true_pos  = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(label_out,tf.bool),tf.cast(y_pred,tf.bool)),tf.int32))
    
    false_pos = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(tf.cast(label_out,tf.bool)),tf.cast(y_pred,tf.bool)),tf.int32))
    true_neg  = tf.reduce_sum(tf.cast(tf.logical_not(tf.logical_or(tf.cast(label_out, tf.bool),\
                                                                   tf.cast(y_pred, tf.bool))),tf.int32))
    false_neg = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not\
                                                     (tf.cast(y_pred,tf.bool)),tf.cast(label_out,tf.bool)),tf.int32))
    print('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF')
    return sig,logits, label_out,true_pos,false_pos,true_neg,false_neg

In [ ]:
np.sum(l2)

In [ ]:
np.sum(l3)

In [ ]:
np.sum(l4)

In [ ]:
np.sum(l5)

In [ ]:

features=classification_net(inputs,tf.ones(32,1),tf.constant(True),None)

In [ ]:
features.get_shape()

In [ ]:
features=tf.squeeze(features,axis=2)

In [ ]:
features=tf.squeeze(features,axis=0)

In [ ]:
features.get_shape()

In [ ]:
#a=np.reshape(np.array([i for i in range(1,61)],dtype=np.float32),(12,5))

num_points=512

num_points1 = 1024 
#features=tf.placeholder(tf.float32,[1,1024,4])
idx=tf.placeholder(tf.int32,[None,num_points,1])

idx1=tf.placeholder(tf.int32,[None,num_points1,1])

label_out = tf.placeholder(tf.int32, [None])
label_out1 = tf.placeholder(tf.int32, [None])
def fn(x):
    return tf.gather_nd(features,x)
windows=tf.map_fn(fn,idx,dtype=tf.float32)
windows_1 = tf.map_fn(fn,idx1,dtype=tf.float32)

In [ ]:
windows_1.get_shape()

In [ ]:
windows_1024=tf.reduce_max(windows,axis=1)
windows_1_1024 = tf.reduce_max(windows_1,axis=1)

In [ ]:
windows_1_1024.get_shape()

In [ ]:
is_training=tf.constant(True)
bn_decay=None

In [ ]:
winex_1.get_shape()

In [ ]:
winex=tf.reshape(windows_1024,shape=[-1,1,1,1024])
winex_1 =  tf.reshape(windows_1_1024,shape=[-1,1,1,1024])
winex.get_shape()
winex = tf.concat([winex, winex_1], axis=0)
label_out2 = tf.concat([label_out,label_out1],axis=0)

In [ ]:
#classification head
d1 = tf_util.conv2d(winex, 512, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='dense1', bn_decay=bn_decay)
d2 = tf_util.conv2d(d1, 256, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='dense2', bn_decay=bn_decay)
logits = tf_util.conv2d(d2, 1, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='out', bn_decay=bn_decay, activation_fn=None)


In [ ]:
logits = tf.squeeze(logits)

In [ ]:
## focal loss
sig = tf.sigmoid(logits)

sig = tf.reshape(tf.squeeze(sig),(-1,1))

func_fl = focal_loss(label_out2,tf.concat([1.0-sig, sig],1), gamma=1.0, alpha=0.01)


loss = tf.reduce_mean(func_fl)


optimizer = tf.train.AdamOptimizer(5*1e-3)

train_op = optimizer.minimize(loss)

In [ ]:
prob = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    index = np.argsort(TRAIN_DATASET.get_center_view_point_set(0)[:,2])
    point_set = TRAIN_DATASET.get_center_view_point_set(0)[index]

    if point_set.shape[0]<2048:
            indexes=np.arange(np.shape(point_set)[0])
            indexes2=np.random.choice(indexes, 2048-np.shape(indexes)[0],replace=True)
            indexes=np.concatenate((indexes,indexes2))
    else:
            indexes=np.random.choice(np.arange(np.shape(point_set)[0]),2048,replace=False)
            
    point_set = np.reshape(point_set[indexes], (1, 2048, 4))
    for j in range(1000):        
        _, x =sess.run([train_op, loss],feed_dict={idx:ind, idx1:ind1, label_out: l.astype(np.int32), label_out1: l1.astype(np.int32), inputs:point_set})
        #print(x)
        #print(c)
        #print(n)
    prob = sess.run([sig],feed_dict={idx:ind, idx1:ind1, label_out: l.astype(np.int32), label_out1: l1.astype(np.int32), inputs:point_set})
    print(prob)

In [ ]:
np.sum(np.array(h)>0.5)
np.sum(l)

In [ ]:
np.unique(ind, axis=0).shape

In [ ]:
with tf.device('/device:GPU:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
# Runs the op.
print(sess.run(c))


In [ ]:
s.shape

In [ ]:
len(set(zip(list(s),list(e))))

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
with tf.Session(config=tf.ConfigProto(allow_soft_placement=False, log_device_placement=True)) as sess:
    print(sess.run(c))

In [ ]:
! which pip

In [ ]:
h